# Table of Contents
1  Initialize and create an interactive map
2  Add basemaps
3  Add WMS and XYZ tile layers
4  Search and add Earth Engine data to layer
5  Search Earth Engine API documentation
6  Use Inspector tool and Drawing tools
7  Convert JavaScripts to Python
8  Use and upload local shapefiles
9  Download a image from google earth engine

In [1]:
# -------------------------------------------------------
# 1 import ee and Initialize the map
# -------------------------------------------------------
import ee, geemap

# ee.Initialize()
Map = geemap.Map()
Map = geemap.Map(center=(39, 110), zoom=4)
Map


Map(center=[39, 110], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [ ]:
# -------------------------------------------------------
# 2 Add basemaps and split Map
# -------------------------------------------------------
# geemap.Map()
Map.add_basemap('HYBRID')

In [ ]:
# -------------------------------------------------------
# 2 Add basemaps and split Map
# -------------------------------------------------------
# Map.split_map(left_layer='SATELLITE', right_layer='Esri.WorldImagery')
# Map
# Map.setCenter(29.68, 116.015,zoom=8)
Map.split_map(
    left_layer='NLCD 2016 CONUS Land Cover', right_layer='NLCD 2001 CONUS Land Cover'
)
Map

In [ ]:
Map = geemap.Map()

In [ ]:
# -------------------------------------------------------
# 3  Add WMS and XYZ tile layers
# -------------------------------------------------------
# del Map
# Map = geemap.Map()
url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'
Map.add_tile_layer(url, name='Google Satellite', attribution='Google')
Map

In [4]:
# -------------------------------------------------------
# 4  Search and add Earth Engine data to layer
# -------------------------------------------------------

# https://developers.google.com/earth-engine/datasets/catalog

image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').select('B3', 'B4', 'B5')
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}
Map.setCenter(-122.1899, 37.5010, 8)      # 按地图视图按经纬度坐标来
Map.addLayer(image, vizParams, 'false color composite')
# bands参数需要一个有且只含三个元素的列表，列表里每一个参数为影像的一个波段的名称
# 用这三个波段来映射到RGB的红绿蓝三种颜色中
# min,max设置显示波段值的颜色范围
# gamma，即用于gamma矫正的一些参数，以矫正图像

In [ ]:
# -------------------------------------------------------
# 5  Search Earth Engine API documentation
# -------------------------------------------------------
#     https://developers.google.com/earth-engine/guides
#     https://code.earthengine.google.com/?accept_repo=users/nclinton/EE101
# -------------------------------------------------------
# 6  Use Inspector tool and Drawing tools
# -------------------------------------------------------
Map.draw_features
# Map.draw_last_feature
feature = Map.draw_last_feature
# roi = feature
roi = feature.geometry()

In [ ]:
# 6  Use Inspector tool and Drawing tools -> example
# image = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filterBounds(roi).sort('CLOUD_COVER').first()
image = ee.ImageCollection("LANDSAT/LC08/C01/T1").filterBounds(roi).sort('CLOUD_COVER').first()
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0.0,
  'max': 30000.0
}
Map.addLayer(image.clip(roi),vizParams, 'landsat8')
# Map.addLayer(image,{'bands':['SR_B5','SR_B4','SR_B3']}, 'landsat9')

In [ ]:
# -------------------------------------
# 7  Convert JavaScripts to Python
# -------------------------------------

js_snippet = """
var dataset = ee.ImageCollection('MODIS/006/MCD12Q1');
var igbpLandCover = dataset.select('LC_Type1');
var igbpLandCoverVis = {
  min: 1.0,
  max: 17.0,
  palette: [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ],
};
Map.setCenter(6.746, 46.529, 6);
Map.addLayer(igbpLandCover, igbpLandCoverVis, 'IGBP Land Cover');
"""
geemap.js_snippet_to_py(js_snippet, add_new_cell=True, import_ee=False, import_geemap=False, show_map=False)

In [ ]:
import geemap

Map = geemap.Map()

dataset = ee.ImageCollection('MODIS/006/MCD12Q1')
igbpLandCover = dataset.select('LC_Type1')
igbpLandCoverVis = {
  'min': 1.0,
  'max': 17.0,
  'palette': [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ],
}
Map.setCenter(6.746, 46.529, 6)
Map.addLayer(igbpLandCover.first().clip(roi), igbpLandCoverVis, 'IGBP Land Cover')
Map

In [ ]:
# --------------------------------------------
# 8  Use and upload local shapefiles
# --------------------------------------------

# local shape file 
f_shp = r'D:\08.专业数据\A1_database\A2_images\00.基础地理数据\stations_2k_dd\stations_2k_gcs.shp' #添加面shp（文件名不能有“_”和中文）

# convert local shape file to ee.feature
shp_ee = geemap.shp_to_ee(f_shp)

# add ee.feature to the map
# Map.addLayer(eeObject, visParams, name, shown, opacity)
Map.addLayer(shp_ee, {}, 'Climate stations')

In [2]:
# --------------------------------------------
# 9  Download a image from google earth engine
# Total request size must be less than or equal to 50331648 bytes
# --------------------------------------------
import os
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').select('B3', 'B4', 'B5')
fidx = image.get('system:index').getInfo()

In [3]:
f_img = r'D:\03.Temporary' + os.sep + fidx + 'd.tif'
# geemap.ee_export_image(image, filename=f_img, scale=100,file_per_band=True)
geemap.download_ee_image(image, f_img, scale=30,  crs='epsg:4326')
# geemap.ee_export_image_to_drive(image, description='landsat8', folder='export', scale=30)

LC08_044034_20140318d.tif: |                                                                                  …

In [ ]:
# --------------------------------------------
# 9  Download a image from google earth engine to google drive
# 
# --------------------------------------------
geemap.ee_export_image_to_drive(image, description=fidx, folder='ee_export', scale=30)